# Opioid

In [1]:
import numpy as np
import gillespy2
from gillespy2.core import Model, Species, Reaction, Parameter, RateRule, AssignmentRule, FunctionDefinition
from gillespy2.core.events import EventAssignment, EventTrigger, Event
from gillespy2.solvers.cpp.ssa_c_solver import SSACSolver
# from gillespy2.solvers.cpp.variable_ssa_c_solver import VariableSSACSolver
# from gillespy2.solvers.numpy.basic_tau_leaping_solver import BasicTauLeapingSolver
# from gillespy2.solvers.numpy.basic_tau_hybrid_solver import BasicTauHybridSolver
# from gillespy2.solvers.numpy.basic_ode_solver import BasicODESolver

In [2]:
class Opioid(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Opioid")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alpha", expression=0.15))
        self.add_parameter(Parameter(name="epsilon", expression=0.8))
        self.add_parameter(Parameter(name="beta_p", expression=0.00266))
        self.add_parameter(Parameter(name="beta_a", expression=0.00094))
        self.add_parameter(Parameter(name="gamma", expression=0.00744))
        self.add_parameter(Parameter(name="zeta", expression=0.2))
        self.add_parameter(Parameter(name="delta", expression=0.1))
        self.add_parameter(Parameter(name="sigma", expression=0.9))
        self.add_parameter(Parameter(name="mu", expression=0.00729))
        self.add_parameter(Parameter(name="mu_prime", expression=0.01159))

        # Species
        self.add_species(Species(name="Susceptible", initial_value=200, mode="discrete"))
        self.add_species(Species(name="Prescribed_User", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Addicted", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Rehab", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Natural_Deaths", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Addicted_Deaths", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="SP", reactants={'Susceptible': 1}, products={'Prescribed_User': 1}, rate=self.listOfParameters["alpha"]))
        self.add_reaction(Reaction(name="SA_a", reactants={'Susceptible': 1}, products={'Addicted': 1}, rate=self.listOfParameters["beta_a"]))
        self.add_reaction(Reaction(name="SA_p", reactants={'Susceptible': 1}, products={'Addicted': 1}, rate=self.listOfParameters["beta_p"]))
        self.add_reaction(Reaction(name="PA", reactants={'Prescribed_User': 1}, products={'Addicted': 1}, rate=self.listOfParameters["gamma"]))
        self.add_reaction(Reaction(name="PS", reactants={'Prescribed_User': 1}, products={'Susceptible': 1}, rate=self.listOfParameters["epsilon"]))
        self.add_reaction(Reaction(name="AR", reactants={'Addicted': 1}, products={'Rehab': 1}, rate=self.listOfParameters["zeta"]))
        self.add_reaction(Reaction(name="RA", reactants={'Rehab': 1}, products={'Addicted': 1}, rate=self.listOfParameters["delta"]))
        self.add_reaction(Reaction(name="RS", reactants={'Rehab': 1}, products={'Susceptible': 1}, rate=self.listOfParameters["sigma"]))
        self.add_reaction(Reaction(name="mu_S", reactants={'Susceptible': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="mu_P", reactants={'Prescribed_User': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="mu_R", reactants={'Rehab': 1}, products={'Susceptible': 1, 'Natural_Deaths': 1}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="mu_prime_A", reactants={'Addicted': 1}, products={'Susceptible': 1, 'Addicted_Deaths': 1}, rate=self.listOfParameters["mu_prime"]))

        # Timespan
        self.timespan(np.linspace(0, 200, 201))

In [3]:
model = Opioid()

In [4]:
solver = SSACSolver(model=model)

In [5]:
def configure_simulation():
    kwargs = {
        "solver":solver,
        "number_of_trajectories":1,
        "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06}
    }
    return kwargs

In [6]:
kwargs = configure_simulation()
results = model.run(**kwargs)

In [7]:
results.plotplotly()